# 01. 합성곱 신경망 (CNN)
> 이미지 분석 등에 주로 사용되는 합성곱 신경망(CNN)에 대해 알아봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 8]
- permalink: /cnn
- exec: colab

### 1. 합성곱 신경망(CNN)이란?

이미지와 같은 데이터들은 2D 이상의 차원을 가집니다. 그러나 우리가 지금까지 배운 다층퍼셉트론(MLP)등의 모델은 이미지와 같은 2차원 데이터를 있는 그대로 처리하지 못합니다. 무슨말인지 알아볼까요? <br><br>

![](https://testerstories.com/files/ai_and_ml/ml-mnist-2d-to-1d.png)
<br>

우리가 기존에 사용하던 다층 퍼셉트론(MLP) 신경망은 데이터 샘플 하나를 벡터로서 입력받습니다. 우리가 붓꽃 예제를 풀때를 생각해봅시다. 하나의 데이터를 [꽃잎길이, 꽃잎너비, 꽃받침길이, 꽃받침너비]와 같이 Rank가 1인 벡터로 입력받아왔습니다. 이렇게 다층 퍼셉트론은 Rank가 1차원인 데이터를 입력받아야하기 때문에 이미지와 같은 높은 차원의 데이터를 1차원으로 평면화 시켜야합니다. 그러나 이렇게 평면화 시켜서 입력하면 이미지의 공간적인 특징이 사라지게 됩니다. 이 문제를 어떻게 해결했을까요? <br><br>

#### 1.1. 고양이의 눈에서 답을 얻다
![](https://i0.wp.com/k.kakaocdn.net/dn/nni6z/btqBD2aVqYg/uYXKGG1VB7LjZpua8OTOwK/img.png?w=750&ssl=1)

1950년대 허블(Hubel)과 비셀(Wiesel)은 고양이의 시각 피질 실험에서 고양이 시야의 한 쪽에 자극을 주었더니 전체 뉴런이 아닌 특정 뉴런만이 활성화되는 것을 발견했습니다. 즉, 눈이 어느 지점을 바라볼 때 뇌의 모든부분이 활성화 되는 것이 아니라 특정 부분만 활성화된다는 것이였습니다. 
<br><br>

이러한 아이디어에 영향을 받은 얀 르쿤(Yann Lecan) 교수는 기존처럼 모든 노드들이 전부 연결 되어있는 기존의 인공신경망이 아닌 눈이 바라보고 있는 곳에 따라서 특정 노드들만 연결되는 획기적인 인공신경망을 고안해 냈습니다. 이전 시간에 배운 RNN이 인간의 '기억' 메커니즘을 컴퓨터로 모델링했다면, CNN은 동물의 '눈'을 컴퓨터로 모델링한 신경망 구조입니다. <br><br>

#### 1.2. 이미지에 대해서

우리는 보통 이미지를 2차원 데이터라고 부르지만, 사실은 이는 정확한 말이 아닙니다. 흑백이미지의 경우는 2차원이 맞지만, 컬러 이미지는 3차원의 차원을 가집니다. 이게 무슨뜻일까요?

![](https://i0.wp.com/k.kakaocdn.net/dn/bbIGmT/btqBGftlKCn/yx8KQzY8N6tu8OZSyIrlTK/img.png?w=750&ssl=1)

컬러 이미지의 경우 가로, 세로말고 '채널'이라는 개념을 가지고 있습니다. 이 채널은 색상을 의미하는데 일반적으로 우리는 컬러 이미지가 R, G, B의 세개의 채널을 가지고 있다고 이야기합니다. 